# Day 1, Session 2: ReAct Pattern in Action

## Building Intelligent Agents with Reasoning + Acting

In this demo, we'll explore how agents combine Large Language Models (LLMs) with tools to solve real-world tasks. We'll use the ReAct (Reasoning + Acting) pattern to process invoices, showing how agents think step-by-step and take actions based on their reasoning.

### What Makes an Agent?

An agent is more than just an LLM - it's a system that:
- **Reasons** about what to do next
- **Acts** by calling tools and functions
- **Observes** the results of those actions
- **Adapts** its approach based on observations

### The ReAct Pattern

The ReAct pattern follows a simple loop:
1. **Thought**: The LLM reasons about the current state
2. **Action**: The LLM selects and executes a tool
3. **Observation**: The result is fed back to the LLM
4. **Repeat** until the task is complete

Let's see this in action with invoice processing!

In [ ]:
# Download real invoice and receipt images first
import requests
import zipfile
import io
import os

# Dropbox shared link for the folder
dropbox_url = "https://www.dropbox.com/scl/fo/m9hyfmvi78snwv0nh34mo/AMEXxwXMLAOeve-_yj12ck8?rlkey=urinkikgiuven0fro7r4x5rcu&st=hv3of7g7&dl=1"

print(f"Downloading real invoice data from: {dropbox_url}")

try:
    response = requests.get(dropbox_url)
    response.raise_for_status()

    # Read the content as a zip file
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # Extract all contents to a directory named 'downloaded_images'
        z.extractall("downloaded_images")

    print("✅ Downloaded and extracted images to 'downloaded_images' folder.")
    
    # List downloaded files
    for root, dirs, files in os.walk("downloaded_images"):
        for file in files:
            print(f"  📄 {os.path.join(root, file)}")

except Exception as e:
    print(f"❌ Error downloading images: {e}")

# Global configuration - Instructor will fill these
OLLAMA_URL = "http://XX.XX.XX.XX"  # Course server IP (port 80 via nginx)
API_TOKEN = "YOUR_TOKEN_HERE"      # Instructor provides token
MODEL = "qwen3:8b"                  # Default model on server

# For testing locally, you can use these values:
# OLLAMA_URL = "http://localhost:11434"  # If running Ollama locally
# API_TOKEN = "test-token"
# MODEL = "llama2"  # Or any model you have locally

## Step 1: Connect to Course LLM Server

First, let's establish connection to our course LLM server and verify it's working.

In [ ]:
import requests
import json
from datetime import datetime, timedelta
import time

# Check server health
def check_server_health():
    """Check if the LLM server is healthy and has models loaded"""
    try:
        response = requests.get(f"{OLLAMA_URL}/health")
        if response.status_code == 200:
            data = response.json()
            print("Server Status:", data.get('status', 'Unknown'))
            print("Models Available:", data.get('models_count', 0))
            return True
    except Exception as e:
        print(f"Error connecting to server: {e}")
    return False

# Create LLM calling function
def call_llm(prompt, model=MODEL):
    """Call the LLM with a prompt and return the response"""
    headers = {
        "Authorization": f"Bearer {API_TOKEN}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": model,
        "prompt": prompt
    }
    
    try:
        response = requests.post(
            f"{OLLAMA_URL}/think",
            headers=headers,
            json=data
        )
        if response.status_code == 200:
            return response.json().get('response', '')
        else:
            return f"Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Error calling LLM: {e}"

# Test connection
print("Checking server connection...")
if check_server_health():
    print("\nTesting LLM with simple prompt...")
    test_response = call_llm("Hello! Please respond with 'I am ready to process invoices.'")
    print("LLM Response:", test_response[:200])  # First 200 chars
else:
    print("\n⚠️ Server not available. Using mock responses for demo.")

## Step 2: Create Mock Invoice Processing Tools

These tools simulate real invoice processing capabilities. In production, these would connect to actual databases and services.

In [ ]:
# Mock Invoice Processing Tools

def extract_invoice_data(invoice_id):
    """Extract data from an invoice (mock implementation)"""
    # Simulate extracting invoice data
    mock_invoices = {
        "INV-2024-001": {
            "invoice_id": "INV-2024-001",
            "vendor": "TechSupplies Co.",
            "amount": 15000.00,
            "currency": "USD",
            "date": "2024-01-15",
            "payment_terms": "Net 30",
            "vat_number": "GB123456789",
            "line_items": [
                {"description": "Laptops", "quantity": 5, "unit_price": 2000},
                {"description": "Software Licenses", "quantity": 10, "unit_price": 500}
            ]
        },
        "INV-2024-002": {
            "invoice_id": "INV-2024-002",
            "vendor": "CloudServices Inc.",
            "amount": 8500.00,
            "currency": "USD",
            "date": "2024-02-01",
            "payment_terms": "Net 60",
            "vat_number": "US987654321"
        }
    }
    
    if invoice_id in mock_invoices:
        return {"status": "success", "data": mock_invoices[invoice_id]}
    else:
        return {"status": "error", "message": f"Invoice {invoice_id} not found"}

def check_payment_terms(terms, amount):
    """Verify payment terms against business rules"""
    # Business rules:
    # - Net 30 for amounts < 10000
    # - Net 60 allowed for amounts > 10000
    # - Net 90 requires special approval
    
    if terms == "Net 90":
        return {"approved": False, "reason": "Net 90 requires CFO approval"}
    elif terms == "Net 60" and amount <= 10000:
        return {"approved": False, "reason": "Net 60 only allowed for amounts > $10,000"}
    elif terms == "Net 30":
        return {"approved": True, "reason": "Standard payment terms"}
    elif terms == "Net 60" and amount > 10000:
        return {"approved": True, "reason": "Extended terms approved for large amount"}
    else:
        return {"approved": False, "reason": f"Unknown payment terms: {terms}"}

def calculate_due_date(invoice_date, payment_terms):
    """Calculate payment due date based on terms"""
    try:
        date = datetime.strptime(invoice_date, "%Y-%m-%d")
        
        # Extract days from payment terms
        if "Net" in payment_terms:
            days = int(payment_terms.split()[1])
            due_date = date + timedelta(days=days)
            return {"due_date": due_date.strftime("%Y-%m-%d"), "days_until_due": days}
        else:
            return {"error": "Invalid payment terms format"}
    except Exception as e:
        return {"error": str(e)}

def search_vendor_history(vendor_name):
    """Search for vendor payment history (mock database)"""
    mock_history = {
        "TechSupplies Co.": {
            "status": "trusted",
            "total_invoices": 45,
            "on_time_payments": 43,
            "average_amount": 12000,
            "last_invoice_date": "2023-12-15"
        },
        "CloudServices Inc.": {
            "status": "new",
            "total_invoices": 2,
            "on_time_payments": 2,
            "average_amount": 7500,
            "last_invoice_date": "2024-01-01"
        },
        "Unknown Vendor": {
            "status": "not_found",
            "message": "No history found for this vendor"
        }
    }
    
    return mock_history.get(vendor_name, mock_history["Unknown Vendor"])

def validate_vat_number(vat_number):
    """Validate VAT number format and checksum (simplified)"""
    # Simple validation - in reality would use EU VIES service
    if vat_number.startswith("GB") and len(vat_number) == 11:
        return {"valid": True, "country": "United Kingdom", "company": "Verified Company"}
    elif vat_number.startswith("US") and len(vat_number) == 11:
        return {"valid": True, "country": "United States", "company": "Verified US Company"}
    else:
        return {"valid": False, "reason": "Invalid VAT number format"}

# Test tools
print("Testing invoice extraction...")
test_invoice = extract_invoice_data("INV-2024-001")
print(f"Invoice data: {json.dumps(test_invoice['data'], indent=2)[:300]}...")

## Step 3: Implement ReAct Agent with Real LLM

Now we create an agent that uses the LLM to reason about what actions to take.

In [ ]:
class ReActAgent:
    def __init__(self, llm_func, tools, max_steps=10):
        self.llm_func = llm_func
        self.tools = tools
        self.max_steps = max_steps
        self.history = []
        
    def think(self, task, observations=""):
        """Use LLM to reason about next action"""
        
        # Build tool descriptions
        tool_descriptions = "\n".join([
            f"- {name}: {func.__doc__}" 
            for name, func in self.tools.items()
        ])
        
        prompt = f"""You are an invoice processing agent using ReAct pattern.

Task: {task}

Previous observations:
{observations}

Available tools:
{tool_descriptions}
- FINISH: Complete with final answer

Think step by step about what to do next. Consider:
1. What information do I already have?
2. What information do I still need?
3. Which tool can help me get that information?

Respond EXACTLY in this format:
THOUGHT: [your reasoning about what to do next]
ACTION: [tool_name or FINISH]
PARAMS: [parameters as JSON or final answer if FINISH]
"""
        
        response = self.llm_func(prompt)
        return self.parse_response(response)
    
    def parse_response(self, response):
        """Parse structured response from LLM"""
        thought = ""
        action = ""
        params = ""
        
        lines = response.strip().split('\n')
        for line in lines:
            if line.startswith("THOUGHT:"):
                thought = line.replace("THOUGHT:", "").strip()
            elif line.startswith("ACTION:"):
                action = line.replace("ACTION:", "").strip()
            elif line.startswith("PARAMS:"):
                params = line.replace("PARAMS:", "").strip()
        
        return {"thought": thought, "action": action, "params": params}
    
    def act(self, action, params):
        """Execute the chosen action"""
        if action == "FINISH":
            return {"type": "final_answer", "result": params}
        
        if action in self.tools:
            try:
                # Parse parameters if they're JSON
                if params.startswith("{"):
                    params_dict = json.loads(params)
                    result = self.tools[action](**params_dict)
                else:
                    # Single parameter
                    result = self.tools[action](params.strip('"'))
                return {"type": "observation", "result": result}
            except Exception as e:
                return {"type": "error", "result": f"Error executing {action}: {str(e)}"}
        else:
            return {"type": "error", "result": f"Unknown action: {action}"}
    
    def run(self, task):
        """Run the agent on a task"""
        observations = ""
        
        print(f"\n📋 TASK: {task}\n")
        print("=" * 80)
        
        for step in range(self.max_steps):
            print(f"\n🔄 Step {step + 1}:")
            
            # Think
            response = self.think(task, observations)
            print(f"\n💭 THOUGHT: {response['thought']}")
            print(f"⚡ ACTION: {response['action']}")
            print(f"📝 PARAMS: {response['params']}")
            
            # Act
            result = self.act(response['action'], response['params'])
            
            if result['type'] == 'final_answer':
                print(f"\n✅ FINAL ANSWER: {result['result']}")
                return result['result']
            else:
                print(f"\n👁️ OBSERVATION: {json.dumps(result['result'], indent=2)[:500]}")
                observations += f"\nStep {step + 1} - {response['action']}: {json.dumps(result['result'])}\n"
        
        print("\n⚠️ Max steps reached without completion")
        return None

# Create agent with our tools
tools = {
    "extract_invoice": extract_invoice_data,
    "check_terms": lambda terms, amount: check_payment_terms(terms, amount),
    "calculate_due": lambda date, terms: calculate_due_date(date, terms),
    "search_vendor": search_vendor_history,
    "validate_vat": validate_vat_number
}

# For demo, use mock LLM if server not available
def mock_llm(prompt):
    """Mock LLM responses for demo purposes"""
    if "Previous observations:" in prompt and "Step 1" not in prompt:
        if "Step 2" in prompt:
            return """THOUGHT: I have the invoice data. Now I need to verify if the payment terms are acceptable for this amount.
ACTION: check_terms
PARAMS: {"terms": "Net 30", "amount": 15000}"""
        elif "Step 3" in prompt:
            return """THOUGHT: The payment terms are approved. Now I should calculate the due date.
ACTION: calculate_due
PARAMS: {"date": "2024-01-15", "terms": "Net 30"}"""
        elif "Step 4" in prompt:
            return """THOUGHT: I have the due date. Let me check the vendor history to assess risk.
ACTION: search_vendor
PARAMS: "TechSupplies Co."""
        else:
            return """THOUGHT: I have all the information needed. The invoice is valid with approved payment terms.
ACTION: FINISH
PARAMS: Invoice INV-2024-001 is approved. Amount: $15,000, Payment terms: Net 30 (approved), Due date: 2024-02-14, Vendor: TechSupplies Co. (trusted vendor with good payment history)"""
    else:
        return """THOUGHT: I need to first extract the invoice data to understand what we're dealing with.
ACTION: extract_invoice
PARAMS: "INV-2024-001"""

# Use real LLM if available, otherwise mock
llm_function = call_llm if check_server_health() else mock_llm

print("\n✅ Agent created and ready!")

## Step 4: Live Execution - Process an Invoice

Watch the agent think through the invoice processing task step by step.

In [ ]:
# Create and run the agent
agent = ReActAgent(llm_function, tools)

# Process an invoice
task = "Process invoice INV-2024-001 and verify it meets our payment policies. Check all relevant information including vendor history and VAT validation."

result = agent.run(task)

## Step 5: Compare Agent vs Simple LLM

Let's see the difference between an agent that can act and a simple LLM that can only describe.

In [ ]:
print("=" * 80)
print("COMPARISON: Agent vs Simple LLM")
print("=" * 80)

# Simple LLM approach
print("\n📱 SIMPLE LLM APPROACH:")
print("-" * 40)

simple_prompt = f"""How would you process invoice INV-2024-001 and verify it meets payment policies?"""

simple_response = """To process invoice INV-2024-001, I would:
1. Extract the invoice data to see the amount and terms
2. Check if the payment terms align with company policies
3. Calculate the due date based on the terms
4. Verify the vendor's payment history
5. Validate the VAT number

However, I cannot actually access the invoice data or execute these checks.
I can only describe the process."""

print("Question:", simple_prompt)
print("\nResponse:", simple_response)

# Agent approach
print("\n\n🤖 AGENT APPROACH:")
print("-" * 40)
print("The agent actually:")
print("✓ Extracted real invoice data: $15,000 from TechSupplies Co.")
print("✓ Verified payment terms: Net 30 approved for this amount")
print("✓ Calculated due date: 2024-02-14")
print("✓ Checked vendor history: Trusted vendor with 95% on-time payment rate")
print("✓ Provided actionable result: Invoice approved for processing")

print("\n💡 KEY DIFFERENCE:")
print("- Simple LLM: Can only describe what it would do")
print("- Agent: Actually executes tools and verifies real data")
print("- Result: Agent provides accurate, actionable information")

## Step 6: Error Handling Demo

Let's see how the agent handles errors and unexpected situations.

In [ ]:
print("=" * 80)
print("ERROR HANDLING DEMONSTRATION")
print("=" * 80)

# Try to process a non-existent invoice
error_task = "Process invoice INV-2024-999 and verify payment terms."

# Create a modified mock LLM for error handling
def error_handling_llm(prompt):
    if "Previous observations:" in prompt:
        if "error" in prompt.lower() and "not found" in prompt.lower():
            return """THOUGHT: The invoice INV-2024-999 was not found. I should report this issue and suggest checking the invoice ID.
ACTION: FINISH
PARAMS: Error: Invoice INV-2024-999 not found in the system. Please verify the invoice ID and try again."""
    return """THOUGHT: I need to extract the invoice data first.
ACTION: extract_invoice
PARAMS: "INV-2024-999"""

# Run agent with error scenario
error_agent = ReActAgent(error_handling_llm, tools, max_steps=3)
error_result = error_agent.run(error_task)

print("\n" + "=" * 80)
print("✅ The agent successfully handled the error by:")
print("1. Attempting to extract the invoice")
print("2. Recognizing the error in the observation")
print("3. Providing a helpful error message")
print("4. Suggesting corrective action")

## Step 7: Performance Metrics

Understanding the performance characteristics of our agent system.

In [ ]:
import time

# Simulate performance metrics
print("=" * 80)
print("PERFORMANCE METRICS")
print("=" * 80)

# Check server metrics if available
def get_server_metrics():
    try:
        response = requests.get(f"{OLLAMA_URL}/metrics")
        if response.status_code == 200:
            return response.json()
    except:
        pass
    return None

metrics = get_server_metrics()

if metrics:
    print("\n📊 Server Metrics:")
    print(f"GPU Memory Used: {metrics.get('gpu', {}).get('memory_used', 'N/A')} MB")
    print(f"GPU Memory Total: {metrics.get('gpu', {}).get('memory_total', 'N/A')} MB")
else:
    print("\n📊 Simulated Metrics:")

# Measure agent execution time
print("\n⏱️ Agent Performance:")

# Time a simple task
start_time = time.time()
test_agent = ReActAgent(mock_llm, tools, max_steps=5)
test_agent.run("Extract and validate invoice INV-2024-001")
execution_time = time.time() - start_time

print(f"\nTotal execution time: {execution_time:.2f} seconds")
print(f"Average time per step: {execution_time/4:.2f} seconds")

# Cost estimation
print("\n💰 Cost Analysis (Estimated):")
print(f"Model: {MODEL}")
print(f"Steps executed: 4")
print(f"Estimated tokens per step: ~500")
print(f"Total tokens: ~2000")
print(f"Estimated cost: ~$0.002 (at $0.001 per 1K tokens)")

print("\n🔍 Optimization Opportunities:")
print("1. Cache vendor history for repeated lookups")
print("2. Batch multiple invoice validations")
print("3. Use smaller model for simple extraction tasks")
print("4. Implement parallel tool execution")

## Key Learnings

### What We Demonstrated:

1. **ReAct Pattern in Action**
   - Thought → Action → Observation loop
   - LLM provides reasoning, tools provide accuracy
   - Each observation informs the next decision

2. **Agent vs Simple LLM**
   - Agents can execute real actions and verify data
   - Simple LLMs can only describe what they would do
   - Agents provide accurate, actionable results

3. **Error Handling**
   - Agents can recognize and recover from errors
   - Graceful degradation when tools fail
   - Clear error messages guide users

4. **Performance Considerations**
   - Each reasoning step requires an LLM call
   - Tool execution adds latency
   - Caching and optimization are important

### Why This Matters:

- **Reliability**: Tools provide ground truth, not hallucinations
- **Flexibility**: Agent adapts based on observations
- **Scalability**: Pattern works for simple to complex workflows
- **Transparency**: Can see and audit the reasoning process

### Next Steps:

In the next session, we'll use LangGraph to:
- Build more complex agent workflows
- Handle parallel tool execution
- Implement state management
- Create production-ready agent systems